Условие: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary': ['70', '80', '60', '90'], 'ManagerId': ['3', '4', 'Null', 'Null']}
      

In [9]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

In [10]:
# Создать SparkSession
spark = SparkSession.builder.appName("Spark SQL Example").getOrCreate()

In [11]:
# Создать DataFrame из словаря данных
data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary': ['70', '80', '60', '90'], 'ManagerId': ['3', '4', 'Null', 'Null']}

In [12]:
df = spark.createDataFrame(pd.DataFrame(data))
df

DataFrame[id: bigint, Name: string, Salary: string, ManagerId: string]

In [13]:
# Заменить Null значения в ManagerId на 0
df = df.withColumn("ManagerId", F.coalesce(df.ManagerId, F.lit(0)))

In [14]:
# Присоединить DataFrame к самому себе, чтобы сопоставить сотрудников с их менеджерами
joined_df = df.alias("employees").join(df.alias("managers"), F.col("employees.ManagerId") == F.col("managers.id"), "left")

In [15]:
# Отфильтровать сотрудников, которые зарабатывают больше своих менеджеров
result_df = joined_df.filter(F.col("employees.Salary") > F.col("managers.Salary"))

In [16]:
# Выбрать столбцы с именами сотрудников
result_df = result_df.select("employees.Name")

In [17]:
# Вывести результат
result_df.show()

+----+
|Name|
+----+
| Joe|
+----+

